In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.stem import SnowballStemmer
import spacy
import nltk
import re
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/BerengerQueune/wild_notebooks/main/Dataset/train.csv")
df.shape

(27481, 4)

Keep only positive and negative tweets (so you exclude neutral). What is the percentage of positive/negative tweets?

In [ ]:
df_positive_negative = df.loc[df["sentiment"].isin(["negative", "positive"])]

df_positive_negative["sentiment"].value_counts(normalize=True)*100

positive    52.447595
negative    47.552405
Name: sentiment, dtype: float64

<b><font color='orange'>The percentage of positive tweets is 52,44% and negative tweets is 47,55%.</font></b>

In [ ]:
# df_positive_negative.reset_index(drop=True,inplace=True)

Create a function you call clean that takes a sentence as parameter (so a str text) and returns a text (str) of tokens after applying a stemmer or a lemmatizer, separated by spaces.

In [ ]:
def clean(sentence):
  newsentence = ""
  nlp = spacy.load('en_core_web_sm')
  sent_tokens = nlp(sentence)
  for token in sent_tokens:   
    newsentence = newsentence + " " + (token.lemma_)
  print(newsentence)

clean("You are better when I am well.")

 -PRON- be well when -PRON- be well .


Get the list of English stopwords from NLTK, and copy it into a stopwordsenglish list. Complete your clean function so that it removes punctuation and stopwords.

In [ ]:
stopwordsenglish = nltk.corpus.stopwords.words("english")
nlp = spacy.load('en_core_web_sm')

def clean(sentence):
  sentence = re.sub(r'[^\w\s]','',str(sentence))
  sentence = nltk.word_tokenize(sentence.lower())
  newsentence = ""
  tokens_clean = []
  for words in sentence:
    if words not in stopwordsenglish:
      tokens_clean.append(words)
  tokens_clean2 = ' '.join([str(item) for item in tokens_clean])
  sent_tokens = nlp(tokens_clean2)
  for token in sent_tokens:
    newsentence = newsentence + " " + (token.lemma_)
  return newsentence

clean("You are better when I am well.")

' better well'

Apply this clean function to the text column of your DataFrame. Store the result in a new clean column in the DataFrame. (The processing may take 2 or 3 minutes)
You should now have a DataFrame that looks like this:

In [ ]:
df_positive_negative["clean"] = df_positive_negative["text"].apply(clean)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_positive_negative

,textID,text,selected_text,sentiment,clean
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad miss san diego
2,088c60f138,my boss is bullying me...,bullying me,negative,boss bullying
3,9642c003ef,what interview! leave me alone,leave me alone,negative,interview leave alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,son could not put release already buy
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive,2 a.m. feeding baby fun smile coo
...,...,...,...,...,...
27475,b78ec00df5,enjoy ur night,enjoy,positive,enjoy ur night
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,wish could come see u denver husband lose job...
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,-PRON- have wonder rake client make clear net...
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,yay good enjoy break probably need hectic wee...


Copy the clean column into a Series X, and the sentiment column into a Series y. Apply a split test-train with the training set size at 0.75 with random_state = 32.

In [ ]:
X = df_positive_negative["clean"]
y = df_positive_negative['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 32, train_size = 0.75)

def classification_models(train, test):
  
    from sklearn.neighbors import KNeighborsClassifier
    model = KNeighborsClassifier(n_neighbors=2)
    model.fit(X_train_CV, y_train)
    print("Score for the KNN Train dataset :", model.score(train, y_train))
    print("Score for the KNN Test dataset :", model.score(test, y_test))

    from sklearn.linear_model import LogisticRegression
    modelLR = LogisticRegression()
    modelLR.fit(X_train_CV, y_train)
    print("Score for the Logistic Regression Train dataset :", modelLR.score(train, y_train))
    print("Score for the Logistic Regression Test dataset :", modelLR.score(test, y_test))

    from sklearn.tree import DecisionTreeClassifier
    modelDTC = DecisionTreeClassifier()
    modelDTC.fit(X_train_CV, y_train)
    print("Score for the Decision Tree Classifier Train dataset :", modelDTC.score(train, y_train))
    print("Score for the Decision Tree Classifier Test dataset :", modelDTC.score(test, y_test))

Apply a CountVectorizer and train classification models.

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train)

X_train_CV = vectorizer.transform(X_train)

X_test_CV = vectorizer.transform(X_test)

classification_models(X_train_CV, X_test_CV)

Score for the KNN Train dataset : 0.8334419817470665
Score for the KNN Test dataset : 0.7191395746761183
Score for the Logistic Regression Train dataset : 0.9565677966101694
Score for the Logistic Regression Test dataset : 0.8618919579564899
Score for the Train dataset : 0.9995110821382008
Score for the Test dataset : 0.8203373258372036


Apply a TfidfVectorizer and train classification models.

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train)

X_train_CV = tfidf.transform(X_train)

X_test_CV = tfidf.transform(X_test)

classification_models(X_train_CV, X_test_CV)

Score for the KNN Train dataset : 0.5945241199478487
Score for the KNN Test dataset : 0.5678318259594232
Score for the Logistic Regression Train dataset : 0.9295958279009127
Score for the Logistic Regression Test dataset : 0.8631141530188218
Score for the Train dataset : 0.9995110821382008
Score for the Test dataset : 0.8188706917624052


Compare the scores, which parameters give the best scores?

<b><font color='orange'>I think the Logistic Regression with a CountVectorizer is giving the best score.</font></b>